In [1]:
import numpy as np
import pandas as pd
import os
import random


CG

In [ ]:
for group in range(1,11):
    cg = pd.read_csv(f"Group_{group:02d}/Group_{group:02d}_CGA.csv")
    segments = pd.read_csv(f"Group_{group:02d}/Group_{group:02d}_Oracle.csv")


    mapped_rows = []

    # columns_to_map = cg.columns.difference(["Begin Time - ss.msec", "End Time - ss.msec"])
    columns_to_map = ["Common Ground"]

    for _, segment_row in segments.iterrows():
        segment_start = segment_row["Start"]
        segment_end = segment_row["End"]

        # edit the next line to change mapping strategy if an utterance can be mapped to multiple other utterances
        overlapping_rows = cg[(cg["End Time - ss.msec"] >= segment_start) & (cg["Begin Time - ss.msec"] <= segment_end)]
        # overlapping_rows = overlapping_rows.sort_values(by="Duration - ss.msec")
        
        if not overlapping_rows.empty:
            mapped_data = {
                "Utterance": segment_row["Utterance"],
                "Start": segment_start,
                "End": segment_end
            }
            
            for col in columns_to_map:
                mapped_data[col] = overlapping_rows.iloc[0][col]
            
            mapped_rows.append(mapped_data)
        else:
            # If there are no overlapping rows, create a mapping with empty values
            mapped_data = {
                "Utterance": segment_row["Utterance"],
                "Start": segment_start,
                "End": segment_end
            }
            for col in columns_to_map:
                mapped_data[col] = None
            mapped_rows.append(mapped_data)

    mapped_df = pd.DataFrame(mapped_rows)
    # mapped_df[["Observation", "Inference", "Statement", "Accept", "Doubt", "Question", "Recommendation", "Absence"]] = 0
    # mapped_df[["Observation", "Inference", "Statement", "Accept", "Doubt", "Recommendation"]] = 0
    # mapped_df[["Observation", "Inference", "Statement", "Accept", "Doubt", "Recommendation", "Absence"]] = 0
    mapped_df[["Statement", "Accept", "Doubt"]] = 0
    for i in range(len(mapped_df)):
        if mapped_df.iloc[i]["Common Ground"]:
            if "STATEMENT" in mapped_df.iloc[i]["Common Ground"]:
                mapped_df.at[i, "Statement"] = 1
            # elif "OBSERVATION" in mapped_df.iloc[i]["CGA"]:
            #     mapped_df.at[i, "Observation"] = 1
            # elif "INFERENCE" in mapped_df.iloc[i]["CGA"]:
            #     mapped_df.at[i, "Inference"] = 1
            elif "ACCEPT" in mapped_df.iloc[i]["Common Ground"]:
                mapped_df.at[i, "Accept"] = 1
            elif "DOUBT" in mapped_df.iloc[i]["Common Ground"]:
                mapped_df.at[i, "Doubt"] = 1
            # elif "QUESTION" in mapped_df.iloc[i]["CGA"]:
            #     mapped_df.at[i, "Question"] = 1
            # elif "RECOMMENDATION" in mapped_df.iloc[i]["CGA"]:
            #     mapped_df.at[i, "Recommendation"] = 1
        # else:
        #     mapped_df.at[i, "Absence"] = 1

    mapped_df = mapped_df.drop("Common Ground", axis=1)



    # mapped_df.sort_values(by="Utterance").to_csv(f"Group_{group:02d}/Group_{group:02d}_CG.csv", index=False)
    # mapped_df.sort_values(by="Utterance").to_csv(f"Group_{group:02d}/Group_{group:02d}_CG_abs.csv", index=False)
    mapped_df.sort_values(by="Utterance").to_csv(f"Group_{group:02d}/Group_{group:02d}_CG_3_gold.csv", index=False)

CPS

In [ ]:
for group in range(1,11):
    cps = pd.read_csv(f"Group_{group:02d}/GROUP_{group:02d}_CPS.csv")
    segments = pd.read_csv(f"Group_{group:02d}/Group_{group:02d}_Oracle.csv")


    mapped_rows = []

    columns_to_map = cps.columns.difference(["Start", "End", "Utterance"])

    for _, segment_row in segments.iterrows():
        segment_start = segment_row["Start"]
        segment_end = segment_row["End"]
        
        overlapping_rows = cps[(cps["End"] >= segment_start) & (cps["Start"] <= segment_end)]
        
        if not overlapping_rows.empty:
            mapped_data = {
                "Utterance": segment_row["Utterance"],
                "Start": segment_start,
                "End": segment_end
            }
            
            for col in columns_to_map:
                mapped_data[col] = overlapping_rows[col].values[0]
            
            mapped_rows.append(mapped_data)
        else:
            # If there are no overlapping rows, create a mapping with empty values
            mapped_data = {
                "Utterance": segment_row["Utterance"],
                "Start": segment_start,
                "End": segment_end
            }
            for col in columns_to_map:
                mapped_data[col] = None
            mapped_rows.append(mapped_data)

    mapped_df = pd.DataFrame(mapped_rows)

    mapped_df.sort_values(by="Utterance").to_csv(f"Group_{group:02d}/GROUP_{group:02d}_CPS_Oracle.csv", index=False, columns=cps.columns)

GAMR

In [ ]:
for group in range(1,11):
    gamr = pd.read_csv(f"GROUP_{group:02d}_GAMR_RB.csv")
    segments = pd.read_csv(f"Group_{group:02d}/Group_{group:02d}_Oracle.csv")    # seg_com = segments.copy().iloc[:,:4]

    mapped_rows = []
    
    mapped_indices = set()

    for _, gamr_row in gamr.iterrows():
        gamr_begin = gamr_row["Begin Time - ss.msec"]
        gamr_end = gamr_row["End Time - ss.msec"]
        
        overlapping_rows = segments[(segments["End"] >= gamr_begin) & (segments["Start"] <= gamr_end)]
        
        if not overlapping_rows.empty:
            max_overlap_row = overlapping_rows.loc[(overlapping_rows["End"] - overlapping_rows["Start"]).idxmax()]
            mapped_row = {
                "Utterance": max_overlap_row["Utterance"],
                "Start": max_overlap_row["Start"],
                "End": max_overlap_row["End"],
                "Participant_1": gamr_row["Participant_1"],
                "Participant_2": gamr_row["Participant_2"],
                "Participant_3": gamr_row["Participant_3"]
            }
            mapped_rows.append(mapped_row)

            mapped_indices.add(max_overlap_row.index[0])
    for idx, oracle_row in segments.iterrows():
        if idx not in mapped_indices:
            mapped_row = {
                "Utterance": oracle_row["Utterance"],
                "Start": oracle_row["Start"],
                "End": oracle_row["End"],
                "Participant_1": "",
                "Participant_2": "",
                "Participant_3": ""
            }
            mapped_rows.append(mapped_row)

    mapped_df = pd.DataFrame(mapped_rows)

    mapped_df = mapped_df.sort_values(by="Utterance")

    mapped_df.to_csv(f"Group_{group:02d}/GROUP_{group:02d}_GAMR_Oracle.csv", index=False)


In [21]:
non_overlapping = []
n_gamr = 0
n_segments = 0
for group in range(1,11):
    gamr = pd.read_csv(f"GAMR/GROUP_{group:02d}_GAMR_RB.csv")
    n_gamr += len(gamr)
    segments = pd.read_csv(f"Group_{group:02d}/Group_{group:02d}_Oracle.csv")
    n_segments += len(segments)

    mapped_rows = []
    
    mapped_indices = set()
    for _, gamr_row in gamr.iterrows():
        gamr_begin = gamr_row["Begin Time - ss.msec"]
        gamr_end = gamr_row["End Time - ss.msec"]
        
        overlapping_rows = segments[(segments["End"] >= gamr_begin) & (segments["Start"] <= gamr_end)]
        
        if len(overlapping_rows) == 0:
            non_overlapping.append(gamr_row)

Actions

In [ ]:
for group in range(1,11):
    actions = pd.read_csv(f"Group_{group:02d}/GROUP_{group:02d}_Actions.csv")
    segments = pd.read_csv(f"Group_{group:02d}/Group_{group:02d}_Oracle.csv")


    mapped_rows = []

    columns_to_map = actions.columns.difference(["Begin Time - ss.msec", "End Time - ss.msec", "Duration - ss.msec"])

    for _, segment_row in segments.iterrows():
        segment_start = segment_row["Start"]
        segment_end = segment_row["End"]
        
        overlapping_rows = actions[(actions["End Time - ss.msec"] >= segment_start) & (actions["Begin Time - ss.msec"] <= segment_end)]
        
        if not overlapping_rows.empty:
            mapped_data = {
                "Utterance": segment_row["Utterance"],
                "Start": segment_start,
                "End": segment_end
            }
            
            for col in columns_to_map:
                mapped_data[col] = overlapping_rows[col].values[0]
            
            mapped_rows.append(mapped_data)
        else:
            # If there are no overlapping rows, create a mapping with empty values
            mapped_data = {
                "Utterance": segment_row["Utterance"],
                "Start": segment_start,
                "End": segment_end
            }
            for col in columns_to_map:
                mapped_data[col] = None
            mapped_rows.append(mapped_data)

    mapped_df = pd.DataFrame(mapped_rows)

    mapped_df.sort_values(by="Utterance").to_csv(f"Group_{group:02d}/GROUP_{group:02d}_Actions_Oracle.csv", index=False)